🌱 인프런 📚 모두의 한국어 텍스트 분석과 자연어처리 with 파이썬 🐍 https://inf.run/FX4TP


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/python-text-analysis/blob/main/0402-klue-dnn-input.ipynb)




## DNN (Deep Neural Network) 으로 텍스트 분류하기


## 라이브러리 로드

In [2]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

In [ ]:
# tensorflow
# TextVectorization, Embedding, GlobalAveragePooling1D
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TextVectorization, Embedding
from tensorflow.keras.layers import GlobalAvgPool1D, Dense, Dropout

## 데이터 미리보기 및 요약

In [ ]:
# 학습, 예측 데이터셋을 불러옵니다.
# pecab 으로 조사, 구두점을 제거한 데이터를 불러옵니다. 
# 전처리 되지 않은 텍스트를 사용해도 상관 없습니다.
# train
# test

In [ ]:
# value_counts()로 분류별 빈도수 확인합니다.


In [ ]:
# 정답(label) 값을 설정합니다.
label_name = "topic_idx"

## label one-hot 형태로 만들기

In [ ]:
# from keras.utils import to_categorical

# get_dummies 를 사용하여 label 값을 one-hot 형태로 만듭니다.
# y

In [ ]:
# value_counts 로 정답 빈도 보기


In [ ]:
# one-hot-encoding 정답 빈도 보기


## 검증용 데이터 만들기와 데이터 누수

* 아래의 경우 데이터 누수(Data Leakage)에 해당됩니다.
* label encoding, one-hot encoding 시 test 데이터 셋 활용하여 encoder를 fit하는 경우
* data scaling 적용 시 test 데이터 셋 활용하여 scaler를 fit하는 경우
* pandas의 get_dummies() 함수를 test 데이터셋에 적용하는 경우
* test 데이터 셋의 결측치 처리 시 test 데이터 셋의 통계 값 활용
* test 데이터 셋을 EDA하여 얻은 인사이트를 통해 학습에 활용하는 경우
* test 데이터 셋을 학습 과정에 사용하는 모든 행위 (test 데이터셋은 추론에만 활용되어야 합니다)
* test 데이터 셋의 데이터 개수 정보를 활용하는 경우 (실제 test 데이터셋은 몇개가 입력으로 들어올 지 모르기 때문)
* 위 예시 외에도 test 데이터 셋이 모델 학습에 활용되는 경우에 Data leakage에 해당됨.
* 출처 : https://dacon.io/competitions/official/236055/talkboard/407731 


In [ ]:
# X 
# X_test

In [ ]:
# train_test_split
# X_train_text, X_valid_text, y_train, y_valid

## TextVectorization

* 매개변수
    * max_tokens: 텍스트에서 고려할 최대 토큰 수를 지정합니다. 가장 빈도가 높은 max_tokens 개의 토큰만 사용됩니다.
    * standardize: 텍스트를 표준화하는 방법을 선택합니다. 'lower_and_strip_punctuation'로 설정하면 소문자로 변환하고 문장부호를 제거합니다.
    * split: 텍스트를 토큰화하는 방법을 선택합니다. 'whitespace'로 설정하면 공백을 기준으로 텍스트가 토큰화됩니다.
    * ngrams: 단일 토큰 이상의 n-그램을 생성하도록 지정합니다. 예를 들어, ngrams=2로 설정하면 바이그램이 생성됩니다.
    * output_mode: 출력 모드를 지정합니다. 'int'로 설정하면 정수로 변환된 토큰이 출력됩니다.
    * output_sequence_length: 출력 시퀀스의 길이를 지정합니다. 이 매개변수를 사용하여 모든 시퀀스를 고정된 길이로 패딩하거나 자를 수 있습니다.
    * pad_to_max_tokens: True로 설정하면 max_tokens으로 시퀀스를 패딩합니다.
    * vocabulary: 사용할 어휘 사전을 지정합니다. 사전이 제공되면 모델은 이 사전에 있는 토큰만 고려합니다.
    * sparse: True로 설정하면 희소 텐서를 출력합니다.
    * ragged: True로 설정하면 불규칙한 길이의 토큰을 나타내는 tf.RaggedTensor를 출력합니다.
    * encoding: 텍스트의 인코딩을 지정합니다.
    * kwargs: 기타 설정을 위한 추가적인 키워드 인수를 받습니다.


* API Document: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
import string
string.punctuation

In [ ]:
# 소문자 변환: 입력된 텍스트를 모두 소문자로 변경합니다.
# HTML 태그 제거: HTML 태그(<br />)를 공백으로 대체합니다.
# 구두점 제거: 모든 구두점을 제거합니다.

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

## MAX_SEQUENCE_LENGTH 
* 단어 수에 대한 빈도 수 고려하기

In [ ]:
# 단어 빈도수의 기술 통계 구하기


In [ ]:
# TextVectorization 레이어 설정
# standardize, max_tokens, output_mode
vocab_size = 10000
MAX_SEQUENCE_LENGTH = 10

# vectorizer

In [ ]:
# X_train

In [ ]:
# X_valid

In [ ]:
# X_test

In [ ]:
# vocab

### 텍스트 길이 시각화

In [ ]:
# df_train

###  vocab index 분포
* 0에 가까울 수록 빈도수가 높음

In [ ]:
# vocab index 분포 : 0에 가까울 수록 빈도수가 높음


## 임베딩 

* https://www.tensorflow.org/text/guide/word_embeddings?hl=ko
* https://damien0x0023.github.io/rnnExplainer/

In [ ]:
# 하이퍼파라미터(모델링할 때 사용자가 직접 세팅해주는 값)을 설정합니다.
# vocab_size는 텍스트 데이터의 전체 단어 집합의 크기입니다.
# embedding_dim는 임베딩 할 벡터의 차원입니다.
# max_length는 패딩의 기준이 됩니다.

embedding_dim = 64

In [ ]:
# X_train.shape, embedding_dim

In [ ]:
# embedding_layer

## GlobalAvgPool1D

* 고정된 출력 크기: GlobalAvgPool1D를 사용하면 입력 텍스트의 길이에 관계없이 고정된 출력 크기를 얻을 수 있습니다. 이는 다양한 길이의 문장을 처리하고 동일한 크기의 임베딩을 얻을 수 있도록 해줍니다.
* 전역 정보 캡처: GlobalAvgPool1D는 각 시퀀스 차원에 대한 평균을 계산합니다. 이로써 각 차원의 전역 정보를 캡처할 수 있습니다. 이는 문장의 중요한 특징을 추출하는 데 도움이 되며, 특히 문장이 길거나 다양한 길이일 때 유용합니다.
* 과적합 방지: GlobalAvgPool1D를 사용하면 모델이 입력의 특정 위치에 과도하게 의존하는 것을 방지할 수 있습니다. 이는 모델을 더욱 일반화되게 만들어주어 새로운 데이터에 대한 성능을 향상시킬 수 있습니다.
* 계산 효율성: GlobalAvgPool1D는 입력의 평균을 계산하므로 모델이 더 적은 파라미터로 텍스트를 처리할 수 있습니다. 이는 모델의 효율성을 높여줍니다.

In [ ]:
# avg_pooling_layer heatmap


## 층 구성
 <img src="https://cs231n.github.io/assets/nn1/neural_net2.jpeg" width="40%">

* 이미지 출처 : https://cs231n.github.io/neural-networks-1/

In [ ]:
# 클래스의 수는 분류될 예측값의 종류입니다.
# 정답값이 one-hot 형태로 인코딩 되어 있기 때문에 정답값의 컬럼의 수가 예측값의 종류가 됩니다.
# n_class

In [ ]:
# Sequential Layer



## 모델 컴파일

In [ ]:
# 여러개 정답 중 하나 맞추는 문제이며, 정답값이 one-hot 형태이기 때문에
# 손실 함수는 categorical_crossentropy를 사용합니다.


## 학습 fit

<img src="https://i.imgur.com/uWkUCu1.png" width="700">

* https://www.tensorflow.org/tensorboard/get_started?hl=ko

In [ ]:
# Clear any logs from previous runs
# !rm -rf ./logs/

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir="logs")
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# 모델 학습을 실행합니다.
# history 

In [ ]:
# 모델 학습의 결과값을 데이터 프레임으로 만들어 확인합니다.
# df_hist

In [ ]:
# 모델 학습 결과을 그래프로 시각화합니다.


## tensorboard

In [ ]:
# docs_infra: no_execute
# %load_ext tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir logs

## 예측

In [ ]:
# predict() 메서드로 모델 예측합니다.
# y_pred

In [ ]:
# numpy.argmax를 이용해 가장 큰 값의 인덱스들을 반환한 값(클래스 예측)을 y_predict에 할당합니다.
# y_predict

## 평가
* 실제 미래의 데이터나 경진대회 데이터는 test 의 정답을 모릅니다. 
* 그렇기 때문에 validation 점수를 평가 점수로 활용합니다.

## 제출하기

In [ ]:
# sample_submission 파일을 불러옵니다.
# Dacon 에 제출할 답안지 양식입니다.
submit = pd.read_csv("data/klue/sample_submission.csv")
submit.head()

In [ ]:
# 정답값 측정을 위해 y_test 변수에 할당
# submit["topic_idx"]

In [ ]:
# valid_accuracy

In [ ]:
file_name = f"data/klue/submit_{valid_accuracy:.5f}.csv"
file_name

In [ ]:
# 잘 저장이 되었는지 확인합니다.
submit.to_csv(file_name, index=False)

In [ ]:
# 잘 저장이 되었는지 확인합니다.
pd.read_csv(file_name)

* dacon에 제출해서 리더보드 확인하기 : https://dacon.io/competitions/official/235747/mysubmission


## 여러 방법을 통해 예측비율을 높여보세요. 
* 아래 항목 외의 기법을 사용해도 됩니다.
 * 전처리 하기
 * 불용어 처리
 * BOW, TF-IDF의 파라메터 변경
 * 분류기의 파라메터 변경
 * 분류기 변경

## 임베딩 프로젝터 사용하기
* 임베딩 프로젝터 : [링크](https://projector.tensorflow.org/?hl=ko&_gl=1*qcvijm*_ga*ODk4NzAxOTgzLjE3MDExNjQyNzM.*_ga_W0YLR4190T*MTcwMTgzNzYzMy42LjEuMTcwMTgzODU2Mi4wLjAuMA..)

In [ ]:
# No such layer: embedding. Existing layers are: ['embedding_2', 등의 메시지가 나온다면
# 컴파일을 여러번 해서 그렇습니다. embedding_2 등으로 변경해 주거나 해당 주피터 노트북의 커널 메뉴에서 재시작을 해주시면 됩니다.
# weights
# vocab


In [ ]:
# 가장 첫 데이터는 패딩으로 이후 데이터 부터 사용
# 파일로 저장 : "metadata.tsv",  sep="\t", index=False, header=False
# df_vocab

In [ ]:
# "vectors.tsv"
# index=False, header=False 로 저장
# df_w

* 임베딩 프로젝터 : [링크](https://projector.tensorflow.org/?hl=ko&_gl=1*qcvijm*_ga*ODk4NzAxOTgzLjE3MDExNjQyNzM.*_ga_W0YLR4190T*MTcwMTgzNzYzMy42LjEuMTcwMTgzODU2Mi4wLjAuMA..)

In [ ]:
# colab 사용시 파일 다운로드
# try:
#     from google.colab import files
#     files.download('vectors.tsv')
#     files.download('metadata.tsv')
# except Exception:
#     pass